<a href="https://colab.research.google.com/github/farhanreynaldo/rethinking/blob/master/rethinking_chapter_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load packages used in this notebook
import os
import json
import shutil
import urllib.request
import pandas as pd

In [2]:
# Install package CmdStanPy
!pip install --upgrade cmdstanpy

Requirement already up-to-date: cmdstanpy in /usr/local/lib/python3.6/dist-packages (0.9.62)


In [3]:
# Install pre-built CmdStan binary
# (faster than compiling from source via install_cmdstan() function)
# https://mc-stan.org/users/documentation/case-studies/jupyter_colab_notebooks_2020.html
# https://github.com/stan-dev/example-models/blob/master/knitr/cloud-compute-2020/CmdStanPy_Example_Notebook.ipynb

tgz_file = 'colab-cmdstan-2.23.0.tar.gz'
tgz_url = 'https://github.com/stan-dev/cmdstan/releases/download/v2.23.0/colab-cmdstan-2.23.0.tar.gz'

urllib.request.urlretrieve(tgz_url, tgz_file)
shutil.unpack_archive(tgz_file)

# Specify CmdStan location via environment variable
os.environ['CMDSTAN'] = './cmdstan-2.23.0'

In [4]:
# Check CmdStan path
from cmdstanpy import cmdstan_path
cmdstan_path()

'./cmdstan-2.23.0'

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel

In [6]:
howell1_url = 'https://raw.githubusercontent.com/pymc-devs/resources/master/Rethinking_2/Data/Howell1.csv'
df = pd.read_csv(howell1_url, sep=';', header=0).loc[lambda df: df['age'] >= 18]

In [21]:
data = df.filter(['height', 'weight']).to_dict(orient='list')
data['n'] = df.shape[0]

In [22]:
%%file m41.stan

data {
  int<lower=1> n;
  vector[n] height;
}
parameters {
  real mu;
  real<lower=0,upper=50> sigma;
}
model {
  height ~ normal(mu, sigma);
  sigma ~ uniform(0, 50);
  mu ~ normal(178, 20);
}

Overwriting m41.stan


In [23]:
m41 = CmdStanModel(stan_file='m41.stan')
m41_fit = m41.sample(data=data)
m41_fit.summary()

INFO:cmdstanpy:compiling stan program, exe file: /content/m41
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /content/m41
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-895.69700,0.022432,0.947414,-897.55800,-895.41000,-894.79000,1783.71,11133.9,1.00059
mu,154.61400,0.006524,0.404907,153.96200,154.60800,155.28300,3851.92,24043.7,1.00017
sigma,7.77025,0.004665,0.286078,7.31543,7.76128,8.24665,3761.24,23477.7,1.00044


In [24]:
%%file m42.stan

data {
  int<lower=1> n;
  vector[n] height;
}

parameters {
  real mu;
  real<lower=0, upper=50> sigma;
}

model {
  height ~ normal(mu, sigma);
  sigma ~ uniform(0, 50);
  mu ~ normal(178, .1);
}

Overwriting m42.stan


In [25]:
m42 = CmdStanModel(stan_file='m42.stan')
m42_fit = m42.sample(data=data)
m42_fit.summary()

INFO:cmdstanpy:compiling stan program, exe file: /content/m42
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /content/m42
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 4
INFO:cmdstanpy:finish chain 3


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-1301.6300,0.023368,1.027400,-1303.740,-1301.310,-1300.6400,1933.09,12449.8,1.002110
mu,177.8620,0.001557,0.101150,177.696,177.863,178.0270,4219.71,27176.4,0.999814
sigma,24.6218,0.016821,0.961026,23.114,24.595,26.2786,3264.05,21021.6,0.999616


In [26]:
%%file m43.stan

data {
    int<lower=1> n;
    real xbar;
    vector[n] height;
    vector[n] weight;
}

parameters {
    real a;
    real<lower=0> b;
    real<lower=0, upper=50> sigma;
}

model {
    vector[n] mu;
    mu = a + b * (weight - xbar);
    height ~ normal(mu, sigma);
    a ~ normal(178, 20);
    b ~ lognormal(0, 1);
    sigma ~ uniform(0, 50);
}

Writing m43.stan


In [27]:
data['xbar'] = df['weight'].mean()

m43 = CmdStanModel(stan_file='m43.stan')
m43_fit = m43.sample(data=data)
m43_fit.summary()

INFO:cmdstanpy:compiling stan program, exe file: /content/m43
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /content/m43
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:start chain 2
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:start chain 3
INFO:cmdstanpy:finish chain 2
INFO:cmdstanpy:start chain 4
INFO:cmdstanpy:finish chain 3
INFO:cmdstanpy:finish chain 4


,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,
lp__,-748.22800,0.026078,1.210370,-750.701000,-747.924000,-746.888000,2154.14,3169.21,1.001340
a,154.60300,0.004202,0.277368,154.148000,154.597000,155.074000,4356.57,6409.46,0.999353
b,0.90301,0.000693,0.042166,0.832069,0.903215,0.973593,3705.77,5451.98,0.999882
sigma,5.10748,0.002882,0.190696,4.806830,5.105560,5.427840,4377.79,6440.67,1.000070


In [28]:
m41.code()

'\ndata {\n  int<lower=1> n;\n  vector[n] height;\n}\nparameters {\n  real mu;\n  real<lower=0,upper=50> sigma;\n}\nmodel {\n  height ~ normal(mu, sigma);\n  sigma ~ uniform(0, 50);\n  mu ~ normal(178, 20);\n}'